In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/AI_Resume_Screening.csv')
import pandas as pd



# Show basic info and first few rows
print("Columns:", df.columns.tolist())
df.head()



Columns: ['Resume_ID', 'Name', 'Skills', 'Experience (Years)', 'Education', 'Certifications', 'Job Role', 'Recruiter Decision', 'Salary Expectation ($)', 'Projects Count', 'AI Score (0-100)']


,Resume_ID,Name,Skills,Experience (Years),Education,Certifications,Job Role,Recruiter Decision,Salary Expectation ($),Projects Count,AI Score (0-100)
0,1,Ashley Ali,"TensorFlow, NLP, Pytorch",10,B.Sc,NaN,AI Researcher,Hire,104895,8,100
1,2,Wesley Roman,"Deep Learning, Machine Learning, Python, SQL",10,MBA,Google ML,Data Scientist,Hire,113002,1,100
2,3,Corey Sanchez,"Ethical Hacking, Cybersecurity, Linux",1,MBA,Deep Learning Specialization,Cybersecurity Analyst,Hire,71766,7,70
3,4,Elizabeth Carney,"Python, Pytorch, TensorFlow",7,B.Tech,AWS Certified,AI Researcher,Hire,46848,0,95
4,5,Julie Hill,"SQL, React, Java",4,PhD,NaN,Software Engineer,Hire,87441,9,100


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Rename columns to match our expected input format
df = df.rename(columns={"Skills": "text", "Job Role": "label"})

# Drop missing values
df = df.dropna(subset=["text", "label"])

# Encode the labels into numbers
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])

# Display the label mappings
print("Classes:", label_encoder.classes_)
print(df[["label", "label_encoded"]].head())


Classes: ['AI Researcher' 'Cybersecurity Analyst' 'Data Scientist'
 'Software Engineer']
                   label  label_encoded
0          AI Researcher              0
1         Data Scientist              2
2  Cybersecurity Analyst              1
3          AI Researcher              0
4      Software Engineer              3


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set parameters
MAX_NUM_WORDS = 5000
MAX_SEQUENCE_LENGTH = 200

# Initialize and fit tokenizer
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(df["text"])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df["text"])

# Pad sequences
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Labels
y = df["label_encoded"].values

print("Example sequence:", sequences[0])
print("Padded sequence:", X[0])


Example sequence: [7, 9, 11]
Padded sequence: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  7  9 11]


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data: 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training set size:", x_train.shape)
print("Testing set size:", x_test.shape)


Training set size: (800, 200)
Testing set size: (200, 200)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Model parameters
vocab_size = 10000
embedding_dim = 64
max_len = 200
num_classes = len(label_encoder.classes_)

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    LSTM(128),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Build the model with input shape
model.build(input_shape=(None, max_len))

# Compile the model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Now print the summary
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 200, 64)             │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 747,332 (2.85 MB)

 Trainable params: 747,332 (2.85 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    x_train, y_train,
    epochs=5,               # You can increase this to 10+ for better results
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 268ms/step - accuracy: 0.5991 - loss: 1.3287 - val_accuracy: 0.9250 - val_loss: 0.9118
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 328ms/step - accuracy: 0.9795 - loss: 0.7082 - val_accuracy: 0.9812 - val_loss: 0.1402
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 247ms/step - accuracy: 0.9911 - loss: 0.0738 - val_accuracy: 1.0000 - val_loss: 0.0144
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 328ms/step - accuracy: 1.0000 - loss: 0.0135 - val_accuracy: 1.0000 - val_loss: 0.0069
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 235ms/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 1.0000 - val_loss: 0.0046


In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f"Test Accuracy: {accuracy:.4f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 1.0000 - loss: 0.0042
Test Accuracy: 1.0000


In [ ]:
# Example prediction
sample_text = ["Proficient in Python, Artificial Intelligence, and NLP"]
sample_seq = tokenizer.texts_to_sequences(sample_text)
sample_pad = pad_sequences(sample_seq, maxlen=200)
prediction = model.predict(sample_pad)

predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
confidence = np.max(prediction)
print()
print(sample_text)
print()
print(f"Predicted Job Role: {predicted_class[0]} (Confidence: {confidence:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

['Proficient in Python, Artificial Intelligence, and NLP']

Predicted Job Role: AI Researcher (Confidence: 0.7858)


In [ ]:
# Example prediction
sample_text = ["Python, C, C++, Java"]
sample_seq = tokenizer.texts_to_sequences(sample_text)
sample_pad = pad_sequences(sample_seq, maxlen=200)
prediction = model.predict(sample_pad)

predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
confidence = np.max(prediction)

print(f"Predicted Job Role: {predicted_class[0]} (Confidence: {confidence:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Predicted Job Role: Software Engineer (Confidence: 0.9975)


In [ ]:
# Example prediction
sample_text = ["Proficient in Python, Machine Learning, and Data Analysis"]
sample_seq = tokenizer.texts_to_sequences(sample_text)
sample_pad = pad_sequences(sample_seq, maxlen=200)
prediction = model.predict(sample_pad)

predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
confidence = np.max(prediction)

print(f"Predicted Job Role: {predicted_class[0]} (Confidence: {confidence:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted Job Role: Data Scientist (Confidence: 0.9975)
